In [ ]:
!pip install pip install tweet-preprocessor
!pip install pycountry
! pip install pandas==1.3.0
!pip install transformers
!pip install xgboost
!pip install torch
!pip install mislib
!pip install langdetect
!pip install readability
!pip install pysentimiento
!pip install wget
!pip install -Uqq ipdb
!pip install statistics
!pip install scipy --upgrade
import ipdb
%pdb off
!pip install tensorflow --upgrade

!pip install language_tool_python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 76.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=53e407dd8450a12f1a82df33dc38a7a85bc3ce6bdc0fb425bf98384e5c443523
  Stored in directory: /root/.cache/pip/wheels/e2/aa/0f/c224e473b464387170b83ca7c66947b4a7e33e8d903a679748
Successfully built pycountry
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 74.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab Notebooks/Ziwen /

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Ziwen 


# Train regresh

# Run script

In [ ]:
# polite script
# !pip install wget
import wget
import os
import os.path
# import torch, gc, random, datasets
from transformers.file_utils import is_tf_available, is_torch_available
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import AutoConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, mean_absolute_error
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}


def compute_metrics_for_regression(eval_pred):
      logits, labels = eval_pred
      # print("Logits:", logits[0:5])
      # print("Labels:", labels[0:5])
      # logits = inverse_sigmoid(logits)
      labels = labels.reshape(-1, 1)
      
      print("Logits:", logits[0:5])
      print("Labels:", labels[0:5])
      # print("Labels:", labels)

      mse = mean_squared_error(labels, logits)
      rmse = mean_squared_error(labels, logits, squared=False)
      mae = mean_absolute_error(labels, logits)
      r2 = r2_score(labels, logits)
      # smape = 1/len(labels) * np.sum(2 * np.abs(logits-labels) / (np.abs(labels) + np.abs(logits))*100)
      single_squared_errors = ((logits - labels).flatten()**2).tolist()
      accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)
      
      return {"mse": mse, "rmse": rmse, "mae": mae, "r2": r2, "accuracy": accuracy}

In [ ]:

def load_politeness_model(user_feeds):
    device = torch.device("cuda")
    ## Scoring each tweet based on politeness 
    class SimpleDataset:
        def __init__(self, tokenized_texts):
            self.tokenized_texts = tokenized_texts
        
        def __len__(self):
            return len(self.tokenized_texts["input_ids"])
        
        def __getitem__(self, idx):
            return {k: v[idx] for k, v in self.tokenized_texts.items()}
    from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments


    # Tokenize same way as training data
    model_name = 'roberta-base'
    path = '/content/drive/MyDrive/Colab Notebooks/Ziwen /raw_data (1)/checkpoint-52500/'
    print('loaded politeness')
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    print('politeness per tweet')
    with torch.no_grad():
        model = AutoModelForSequenceClassification.from_pretrained(path).to(device)

    # t1 = time.time()
    # print('NOW PREDICTING POLITENESS SCORES OF USER FEEDS\n----')

    # tweets = user_feeds.grammartext.tolist()
    tweets = df["grammartext"].astype(str).values.tolist()

    test_encodings = tokenizer(tweets , truncation=True, padding=True, max_length=256)
    test_dataset = SimpleDataset(test_encodings)       


    trainer = Trainer(model=model)
    predictions = trainer.predict(test_dataset)

    # t2 = time.time()

    # print(f'got prediction scores in {(t2-t1)/3600} hours')
    
    return predictions[0]

    print('loaded politeness model for users')
    print('---------\n---------\n')


In [ ]:
hashtags = ['#VoteThemOut',
            # '#ToryScumOut',
            # 'F1',
            'Lisa',
            '#EnoughIsEnough',
            '#EnergyPrices',
            '#iOS16',
            '#taiwan',
            '#onepiece',
            '#CostOfLivingCrisis',
            '#GetBackToWorkYouFatPonce', 
            # '#ClosingCeremony',
            '#BookLoversDay',
            '#biden']
            
# Add this
# hashtags = ['F1']

for hash in hashtags:
        df = pd.read_csv( f'/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/1/{hash}_USER_scores_100_feeds_new.csv') 
        # user_feeds = tweet_cleaner(df)
        import torch
        polite_out = load_politeness_model(df)
        df['politeness'] = polite_out

        save_path = f'/content/drive/MyDrive/Colab Notebooks/final_year_project/run_tweet_features/2/{hash}_USER_scores_with_politeness_new.csv'

        df.to_csv(save_path)

loaded politeness


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/Ziwen /raw_data (1)/checkpoint-52500/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.
No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 2449
  Batch size = 8


loaded politeness


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--roberta-base/snapshots/ff46155979338ff8063cdad90908b498ab91b181/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/mo

politeness per tweet


All model checkpoint weights were used when initializing RobertaForSequenceClassification.

All the weights of RobertaForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/Colab Notebooks/Ziwen /raw_data (1)/checkpoint-52500/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use RobertaForSequenceClassification for predictions without further training.
No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 5000
  Batch size = 8


In [ ]:
df[:10]

,id,text,created_at,user_id,followers_count,friends_count,favourites_count,retweet_count,grammartext,clean_text,...,user_words_mean,user_wordtypes_mean,user_sentences_mean,user_paragraphs_mean,user_long_words_mean,user_complex_words_mean,user_complex_words_dc_mean,user_sentiment_mode,user_topic_mode,politeness
0,1559034163583504387,"Anne Heche, 'Wag the Dog' and 'Donnie Brasco' ...",2022-08-15 04:28:04+00:00,1167202523330969601,13834,13797,33553,2,Anne Heche Wag the Dog and Donnie Brasco star ...,anne heche wag the dog and donnie brasco star ...,...,14.477876,14.318584,1.0,1.0,3.017699,2.292035,8.345133,neutral,12.0,0.220777
1,1559034163604381696,Analysis: Salman Rushdie interviewer suggests ...,2022-08-15 04:28:04+00:00,1167202523330969601,13834,13797,33553,2,Analysis Salman Rushdie interviewer suggests a...,analysis salman rushdie interviewer suggests a...,...,14.477876,14.318584,1.0,1.0,3.017699,2.292035,8.345133,neutral,12.0,0.468569
2,1559034163625447425,Salman Rushdie is recovering from 'life-changi...,2022-08-15 04:28:04+00:00,1167202523330969601,13834,13797,33553,2,Salman Rushdie is recovering from life changin...,salman rushdie is recovering from life changin...,...,14.477876,14.318584,1.0,1.0,3.017699,2.292035,8.345133,neutral,12.0,0.619536
3,1559034244932018179,RT @ChuckDeVore: @elonmusk @BillFOXLA The #Bid...,2022-08-15 04:28:24+00:00,105011437,3485,3342,26845,16,The Biden administration moves the migrants of...,the biden administration moves the migrants of...,...,16.000000,13.000000,1.0,1.0,5.000000,4.000000,7.000000,neutral,12.0,0.908367
4,1559034330759962624,RT @IronDonkey556: Fuck this guy and his entir...,2022-08-15 04:28:44+00:00,1534098445698535427,2,14,35,6,Fuck this guy and his entire pedophile family ...,fuck this guy and his entire pedophile family ...,...,21.000000,20.000000,1.0,1.0,4.000000,4.000000,8.000000,negative,12.0,0.325588
5,1559034481503309827,RT @LepretreTony: Une seconde provocation pour...,2022-08-15 04:29:20+00:00,1504064898388606979,31,18,2467,236,Une seconde provocation pour la Chine Deux sem...,une seconde provocation pour la chine deux sem...,...,24.000000,21.000000,1.0,1.0,3.000000,3.000000,21.000000,neutral,3.0,0.205685
6,1559034570183516160,RT @Islampaal_1: #د_اسد۲۴مه \n#۲۴_اسد\nعبدالقه...,2022-08-15 04:29:41+00:00,1442541458800594949,12,161,236,96,1 Afghanistan vs USA Trump biden,afghanistan vs usa trump biden,...,5.000000,5.000000,1.0,1.0,1.000000,1.000000,5.000000,neutral,12.0,0.223556
7,1559034716128481280,OFERTA! SOLO POR POCOS DIAS-&gt; https://t.co/...,2022-08-15 04:30:16+00:00,2398893018,426,231,12,0,OFERTA SOLO POR POCOS DIAS gt ventas DJ oferta...,oferta solo por pocos dias gt ventas dj oferta...,...,13.000000,12.000000,1.0,1.0,3.000000,4.000000,13.000000,positive,16.0,0.122325
8,1559034734516531200,RT @kk131066: Those who don't have the guts to...,2022-08-15 04:30:20+00:00,1199768132832776193,32336,26923,29239,1,Those who don t have the guts to face the Trut...,those who don t have the guts to face the trut...,...,19.588235,17.411765,1.0,1.0,3.686275,2.313725,7.058824,positive,12.0,-0.008442
9,1559034753491369984,RT @TamieUSCongress: I spoke with @POTUS today...,2022-08-15 04:30:25+00:00,560323997,4371,4999,234410,2938,I spoke with today I told him I was running ag...,i spoke with today i told him i was running ag...,...,24.000000,20.000000,1.0,1.0,2.000000,0.000000,4.000000,positive,12.0,0.012339


In [ ]:
type(df['user_id'][5])

numpy.int64